In [1]:
pip install tensorflow_federated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.9 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 87.7 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow-compression to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 MB 23.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.6/721.6 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import cv2
import random
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.utils import to_categorical, plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, regularizers, optimizers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-04-18 14:53:05.538857: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 14:53:05.538935: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 14:53:05.538995: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# Check if GPU is available
device = '/GPU:0' if tf.config.experimental.list_physical_devices('GPU') else '/CPU:0'
# Define paths
HAM_PATH_PART1 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1"
HAM_PATH_PART2 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2"
METADATA_PATH = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv"

def load_data_ham(path, metadata_path):
    print("Loading data from:", path)
    data = []
    metadata = pd.read_csv(metadata_path)
    for img_file in os.listdir(path):
        img_id = img_file.split('.')[0]
        metadata_row = metadata[metadata['image_id'] == img_id]
        if not metadata_row.empty:
            lesion_type = metadata_row.iloc[0]['dx']
            img_path = os.path.join(path, img_file)
            img_resize = process_image(img_path)
            data.append([img_resize, lesion_type])
    return data

def process_image(img_path, target_size=(64, 48)):
    img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img_resize = cv2.resize(img_array, target_size)
    img_normalize = img_resize.astype(np.float32) / 255.0
    return img_normalize

with tf.device(device):
    ham_data_part1 = load_data_ham(HAM_PATH_PART1, METADATA_PATH)
    ham_data_part2 = load_data_ham(HAM_PATH_PART2, METADATA_PATH)
all_ham_data = ham_data_part1 + ham_data_part2
column_names = ['image_data', 'skin_disease']
ham_df = pd.DataFrame(all_ham_data, columns=column_names)
print("Structure of the merged HAM dataset:")
print(ham_df.head())
label_encoder = LabelEncoder()
ham_df['skin_disease'] = label_encoder.fit_transform(ham_df['skin_disease'])
print(ham_df.head())
df=pd.DataFrame(ham_df)

Loading data from: /kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1
Loading data from: /kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2
Structure of the merged HAM dataset:
                                          image_data skin_disease
0  [[0.68235296, 0.6666667, 0.6784314, 0.6509804,...           nv
1  [[0.5568628, 0.5803922, 0.5764706, 0.58431375,...           nv
2  [[0.6666667, 0.6431373, 0.67058825, 0.7058824,...           nv
3  [[0.54509807, 0.53333336, 0.5254902, 0.4862745...          bkl
4  [[0.5764706, 0.5686275, 0.5647059, 0.6039216, ...          bkl
                                          image_data  skin_disease
0  [[0.68235296, 0.6666667, 0.6784314, 0.6509804,...             5
1  [[0.5568628, 0.5803922, 0.5764706, 0.58431375,...             5
2  [[0.6666667, 0.6431373, 0.67058825, 0.7058824,...             5
3  [[0.54509807, 0.53333336, 0.5254902, 0.4862745...             2
4  [[0.5764706, 0.5686275, 0.5647059, 0.6039216, ...             2

In [4]:
import random

num_client = 10

df["client"] = ["client_{}".format(random.randint(1, num_client)) for _ in range(df.shape[0])]

In [5]:
client_id_colname = 'client'

client_ids = df[client_id_colname].unique()

train_client_ids = pd.DataFrame(client_ids).sample(frac=0.8).values.ravel().tolist()
test_client_ids = [x for x in client_ids if x not in train_client_ids]

In [6]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [7]:
features ="image_data"

In [8]:
from collections import OrderedDict
import tensorflow as tf
import numpy as np
NUM_EPOCHS = 1
SHUFFLE_BUFFER = 100
def create_tf_dataset_for_client_fn(client_id):
    client_data = dataframe[dataframe[client_id_colname] == client_id]
    client_data_dict = OrderedDict()
    client_data_dict["image_data"] = np.array(client_data['image_data'].values.tolist(), dtype="float32")
    client_data_dict["skin_disease"] = np.array(client_data['skin_disease'].values.tolist(), dtype="int32")

    dataset = tf.data.Dataset.from_tensor_slices(client_data_dict)
    dataset = dataset.shuffle(SHUFFLE_BUFFER).batch(1).repeat(NUM_EPOCHS)
    return dataset

In [9]:
import tensorflow_federated as tff
dataframe = train_df
train_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
    client_ids=train_client_ids,
    serializable_dataset_fn=create_tf_dataset_for_client_fn)

dataframe = test_df
test_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
    client_ids=test_client_ids,
    serializable_dataset_fn=create_tf_dataset_for_client_fn)

In [10]:
train_data.element_type_structure

OrderedDict([('image_data',
              TensorSpec(shape=(None, 48, 64), dtype=tf.float32, name=None)),
             ('skin_disease',
              TensorSpec(shape=(None,), dtype=tf.int32, name=None))])

In [11]:
import collections
NUM_EPOCHS = 1
BATCH_SIZE = 10
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10
def preprocess(dataset):
    def batch_format_fn(element):
        return collections.OrderedDict(x=tf.reshape(element['image_data'], [-1,32,32,3]),
                                       y=tf.reshape(element['skin_disease'], [-1, 1]))
    return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [12]:
from tqdm import tqdm

def make_federated_data(client_data, client_ids):
    return [preprocess(client_data.create_tf_dataset_for_client(x)) for x in tqdm(client_ids)]

In [13]:
NUM_CLIENTS = len(np.unique(train_df[client_id_colname]))

sample_clients = train_data.client_ids[0:NUM_CLIENTS]

federated_train_data = make_federated_data(train_data, sample_clients)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))

100%|██████████| 8/8 [00:00<00:00, 41.46it/s]

Number of client datasets: 8
First dataset: <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))])>


In [14]:
import tensorflow as tf

def create_keras_model(input_shape=(32, 32, 3), num_classes=7):
    base_model = tf.keras.applications.VGG19(include_top=False, input_shape=input_shape,weights='imagenet')

    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = base_model(x, training=True)  # All layers are trainable
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    return model


model = create_keras_model()
model.summary()


80134624/80134624 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 tf.math.truediv (TFOpLambd  (None, 32, 32, 3)         0         
 a)                                                              
                                                                 
 tf.math.subtract (TFOpLamb  (None, 32, 32, 3)         0         
 da)                                                             
                                                                 
 vgg19 (Functional)          (None, 1, 1, 512)         20024384  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                      

In [15]:
example_dataset = train_data.create_tf_dataset_for_client(train_data.client_ids[0])

example_element = next(iter(example_dataset))

example_element['skin_disease'].numpy()

array([2], dtype=int32)

In [16]:
preprocessed_example_dataset = preprocess(example_dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

In [17]:
preprocessed_example_dataset.element_spec

OrderedDict([('x',
              TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None)),
             ('y', TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))])

In [18]:
from keras.metrics import Recall, Precision

def model_fn():
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=preprocessed_example_dataset.element_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [19]:
iterative_process = tff.learning.algorithms.build_weighted_fed_prox(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.001),
    proximal_strength=0.1)

In [20]:
str(iterative_process.initialize.type_signature)

'( -> <global_model_weights=<trainable=<float32[3,3,3,64],float32[64],float32[3,3,64,64],float32[64],float32[3,3,64,128],float32[128],float32[3,3,128,128],float32[128],float32[3,3,128,256],float32[256],float32[3,3,256,256],float32[256],float32[3,3,256,256],float32[256],float32[3,3,256,256],float32[256],float32[3,3,256,512],float32[512],float32[3,3,512,512],float32[512],float32[3,3,512,512],float32[512],float32[3,3,512,512],float32[512],float32[3,3,512,512],float32[512],float32[3,3,512,512],float32[512],float32[3,3,512,512],float32[512],float32[3,3,512,512],float32[512],float32[512,7],float32[7]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64,float32[3,3,3,64],float32[64],float32[3,3,64,64],float32[64],float32[3,3,64,128],float32[128],float32[3,3,128,128],float32[128],float32[3,3,128,256],float32[256],float32[3,3,256,256],float32[256],float32[3,3,256,256],float32[256],float32[3,3,256,256],float32[256],float32[3,3,

In [21]:
state = iterative_process.initialize()

I0000 00:00:1713452186.722306     232 session_provider.cc:108] Found devices: [/physical_device:CPU:0]
I0000 00:00:1713452186.722344     232 session_provider.cc:124] Skipping device: [/physical_device:CPU:0]
I0000 00:00:1713452186.722355     232 session_provider.cc:127] Found:
	0 GPUs
	0 TPUS
in total


In [22]:
NUM_ROUNDS = 10
train_state=state
for round_num in range(0, NUM_ROUNDS):
  result = iterative_process.next(train_state, federated_train_data)
  train_state = result.state
  train_metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

round  0, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.60401005), ('loss', 4.781272), ('num_examples', 1596), ('num_batches', 164)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.65037596), ('loss', 5.635274), ('num_examples', 1596), ('num_batches', 164)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.65037596), ('loss', 5.635275), ('num_examples', 1596), ('num_batches', 164)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('final

In [23]:
evaluation_process = tff.learning.algorithms.build_fed_eval(model_fn)

In [24]:
evaluation_state = evaluation_process.initialize()
model_weights = iterative_process.get_model_weights(train_state)
evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

In [25]:
federated_test_data = make_federated_data(test_data, test_client_ids)

len(federated_test_data), federated_test_data[0]

100%|██████████| 2/2 [00:00<00:00, 52.55it/s]


(2,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))])>)

In [26]:
evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

In [27]:
str(evaluation_output.metrics)

"OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('sparse_categorical_accuracy', 0.69287467), ('loss', 4.9502754), ('num_examples', 407), ('num_batches', 41)])), ('total_rounds_metrics', OrderedDict([('sparse_categorical_accuracy', 0.69287467), ('loss', 4.9502754), ('num_examples', 407), ('num_batches', 41)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])"